# シンプルな11位の解法【詳細】
# very simple 11th place solution【Details】

このNotebookではPseudo Labeling(疑似ラベリング)について説明しています。11位の解法の概要についてはこちらのNotebookを参照してください。  
【日本語&English】TPS Feb 11th place solution  
https://www.kaggle.com/maostack/english-tps-feb-11th-place-solution
  
This Notebook describes Pseudo Labeling, see this Notebook for an overview of the 11th solution.  
【日本語&English】TPS Feb 11th place solution  
https://www.kaggle.com/maostack/english-tps-feb-11th-place-solution

## 疑似ラベリング / Pseudo Labeling
半教師あり学習の手法の一つ / One of the methods of semi-supervised learning  
  
疑似ラベリングは2段階の構成になっている。  
まず、何らかの予測モデルを用意する（今回はLightGBM）。  
第1段階では、モデルを学習させた後、普通にテストデータに対して予測を行う。その予測値をテストデータに対する疑似ラベルとする。つまり、テストデータに対する予測値を疑似的に目的変数(label・target)として扱う。  
第2段階では、"もともとの学習データにテストデータを合体させたもの"を学習データとして用いて、テストデータに対する予測を行う。  
  
The pseudo labeling consists of two steps.  
First, prepare some kind of prediction model (LightGBM in this case).  
In the 1st stage, we train the model and make a prediction for the test data. The predicted value is used as a pseudo-label for the test data. In other words, the predicted value for the test data is treated as a pseudo target variable (label/target).  
In the 2nd stage, predictions are made for the test data using the "original training data combined with the test data" as the training data. 

In [ ]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.simplefilter("ignore")

別のNotebookで既に前処理をしたデータをtrain, testとして読み込んでいます。  
前処理として行ったことは、  
・targetが外れ値の行を除外(targetが4より小さい行を除外)  
・変数"cat6"について"G"は学習データにしか存在しない(テストデータで値がGをとるデータが存在しない)ので、cat6の値がGの行を除外  
・カテゴリ変数に対するLabel Encoding  
・cont列に対するRankGauss変換  
です。最後のRankGauss変換は、決定木系のモデルには影響を与えないのでしなくてもいいのですが一応しておきました。  
除外後の学習データのデータ数は299963になりました。37行減った。テストデータの数は変わっていない。  
  
The data that has already been preprocessed in another Notebook is loaded as train and test.  
What we did as preprocessing was  
・Exclude rows where target is an outlier (exclude rows where target is less than 4)  
・For the variable "cat6", "G" exists only in the training data (there is no data that takes the value G in the test data), so the line with the value G in cat6 is excluded.  
・Label Encoding for categorical variables  
・RankGauss transform for cont columns  
The RankGauss transformation is not necessary because it does not affect the decision tree model, but I did it just in case.  
After preprocessing, the number of data in the training data is now 299963. 37 rows have been reduced. The number of test data has not changed.

In [ ]:
train = pd.read_csv("../input/tps-feb-eda-fe/train_data.csv")
test = pd.read_csv("../input/tps-feb-eda-fe/test_data.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
cat_columns = [f"cat{i}" for i in range(10)]

In [ ]:
X = train.drop(["target"], axis=1)
X_test = test
y = train.target

print(X.shape)
print(X_test.shape)
print(y.shape)

In [ ]:
X

In [ ]:
X_test

In [ ]:
y

In [ ]:
from sklearn.model_selection import KFold
import lightgbm as lgb

SEED = 8970365

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=SEED)

In [ ]:
# パラメータの値は他の方のNotebookを参考にしました。感謝します。
# The value of the parameter was taken from another person's Notebook.
# I appreciate it.

params_lgb = {
    "task": "train",
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "learning_rate": 0.003899156646724397,
    "num_leaves": 63,
    "max_depth": 99,
    "feature_fraction": 0.2256038826485174,
    "bagging_fraction": 0.8805303688019942,
    "min_child_samples": 290,
    "reg_alpha": 9.562925363678952,
    "reg_lambda": 9.355810045480153,
    "max_bin": 882,
    "min_data_per_group": 127,
    "bagging_freq": 1,
    "cat_smooth": 96,
    "cat_l2": 19,
    "verbosity": -1,
    "bagging_seed": SEED,
    "feature_fraction_seed": SEED,
    "seed": SEED
}

まず普通にテストデータに対して予測を行う。  
First, make a prediction for the test data as usual.

In [ ]:
# 予測値を格納するdf
# df to store the predicted value
preds_lgb = pd.DataFrame()

X[cat_columns] = X[cat_columns].astype("category")
X_test[cat_columns] = X_test[cat_columns].astype("category")

for k, (tr_id, vl_id) in enumerate(kf.split(X, y)):
    print("="*50)
    print(f"               KFold{k+1}")
    print("="*50)
    
    X_train, X_val = X.iloc[tr_id, :], X.iloc[vl_id, :]
    y_train, y_val = y.iloc[tr_id], y.iloc[vl_id]
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    
    model_lgb = lgb.train(params=params_lgb,
                          train_set=lgb_train,
                          valid_sets=lgb_val,
                          num_boost_round=100000,
                          early_stopping_rounds=200,
                          verbose_eval=1000)
    
    pred_lgb = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)
    pred_lgb = pd.DataFrame(pred_lgb)
    
    # 予測値を横に連結していく
    # Concatenate the predictions horizontally
    preds_lgb = pd.concat([preds_lgb, pred_lgb], axis=1)

In [ ]:
preds_lgb

In [ ]:
# 平均を計算して、テストデータに対する疑似ラベルとする
# Calculate the mean and use it as a pseudo labels for the test data

label = preds_lgb.mean(axis=1)
label

In [ ]:
# もともとの学習データX, yにテストデータと疑似ラベルを縦に連結する。
# これを新たな学習データとする
# Concatenate the test data and pseudo labels to the original training data X, y.
# Make this the new training data.

X = pd.concat([X, X_test], axis=0).reset_index(drop=True)
y = pd.concat([y, label], axis=0).reset_index(drop=True)

print("X.shape: ", X.shape)
print("y.shape: ", y.shape)

In [ ]:
X

In [ ]:
y

In [ ]:
# 最終予測値を格納するdf
# df to store the final prediction
preds_lgb = pd.DataFrame()

X[cat_columns] = X[cat_columns].astype("category")
X_test[cat_columns] = X_test[cat_columns].astype("category")

for k, (tr_id, vl_id) in enumerate(kf.split(X, y)):
    print("="*50)
    print(f"               KFold{k+1}")
    print("="*50)
    
    X_train, X_val = X.iloc[tr_id, :], X.iloc[vl_id, :]
    y_train, y_val = y.iloc[tr_id], y.iloc[vl_id]
    
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_val = lgb.Dataset(X_val, y_val)
    
    model_lgb = lgb.train(params=params_lgb,
                          train_set=lgb_train,
                          valid_sets=lgb_val,
                          num_boost_round=100000,
                          early_stopping_rounds=200,
                          verbose_eval=1000)
    
    pred_lgb = model_lgb.predict(X_test, num_iteration=model_lgb.best_iteration)
    pred_lgb = pd.DataFrame(pred_lgb)
    preds_lgb = pd.concat([preds_lgb, pred_lgb], axis=1)

# Submission

In [ ]:
submission = pd.read_csv("../input/tabular-playground-series-feb-2021/sample_submission.csv")

#　予測値の平均を計算して、最終的な予測値とする
# Calculate the average of the predictions to get the final prediction.
pred = preds_lgb.mean(axis=1)
submission.target = pred

submission.head()

In [ ]:
submission.to_csv("submission_pseudo_lgb_5.csv", index=False)

このNotebookと同じ予測方法で、パラメータの値を変えたり、シード値を変えたり、early stopping roundsを変えたりなどをしながら、複数の予測提出ファイルを作る。最後にそれらをアンサンブルする。  
以上が11位の解法です。
  
Using the same prediction method as in this Notebook, create multiple prediction submission files, changing the paramters, seed value and early stopping rounds etc.. Finally, ensemble them.  
This is the 11th solution.